# Explore and cluster the neighborhoods in Toronto

Import the coordinates

In [1]:
import urllib.request
import pandas as pd
import requests
from bs4 import BeautifulSoup
#get the data and create the first dataframe
url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
req = requests.get(url)
soup = BeautifulSoup(req.content, 'html.parser')

table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)


df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [2]:
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [3]:
#get the coordinates
df1=pd.read_csv("C:\\Users\\gkiou\\Downloads\\Geospatial_Coordinates.csv")
df1.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [4]:
df1.rename(columns={'Postal Code' : 'PostalCode'},inplace=True)
df1.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [5]:
#Create the dataframe with the coordinates by merging df1 & df2
df2=pd.merge(left=df,right=df1,on='PostalCode')
df2.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


In [6]:
import matplotlib.cm as cm
import matplotlib.colors as colors

#import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests

usage: conda-script.py [-h] [-V] command ...
conda-script.py: error: unrecognized arguments: # uncomment this line if you haven't completed the Foursquare API lab
usage: conda-script.py [-h] [-V] command ...
conda-script.py: error: unrecognized arguments: # uncomment this line if you haven't completed the Foursquare API lab


In [7]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
tor_lat = location.latitude
tor_lon  = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(tor_lat, tor_lon))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [8]:
map_toronto = folium.Map(location=[tor_lat, tor_lon], zoom_start=10)

# add markers to map
for lat, lng, label in zip(df2['Latitude'], df2['Longitude'], df2['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [9]:
CLIENT_ID = '24X5BG4TZOBYQQUIMFNJ0T0004WE21TAUWAFUPWITH0TJZM3' # your Foursquare ID
CLIENT_SECRET = 'GKOPTZRZYZTIB42HVML1DRMN5IKCGFFTJYWSBLLWATXUUNFC' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 24X5BG4TZOBYQQUIMFNJ0T0004WE21TAUWAFUPWITH0TJZM3
CLIENT_SECRET:GKOPTZRZYZTIB42HVML1DRMN5IKCGFFTJYWSBLLWATXUUNFC


Get the first neighborhood's name

In [10]:
df2['Neighborhood'].iloc[0]

'Parkwoods'

Get the neighborhood's longitude and latitude

In [11]:
neighborhood_latitude = df2.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df2.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df2.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Parkwoods are 43.7532586, -79.3296565.


Top 100 venues that are in Parkwoods within a radius of 500  meters

In [12]:
LIMIT=100
radius=500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=24X5BG4TZOBYQQUIMFNJ0T0004WE21TAUWAFUPWITH0TJZM3&client_secret=GKOPTZRZYZTIB42HVML1DRMN5IKCGFFTJYWSBLLWATXUUNFC&v=20180605&ll=43.7532586,-79.3296565&radius=500&limit=100'

In [13]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '609fc6648220f7718b469761'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Parkwoods - Donalda',
  'headerFullLocation': 'Parkwoods - Donalda, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 2,
  'suggestedBounds': {'ne': {'lat': 43.757758604500005,
    'lng': -79.32343823984928},
   'sw': {'lat': 43.7487585955, 'lng': -79.33587476015072}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e8d9dcdd5fbbbb6b3003c7b',
       'name': 'Brookbanks Park',
       'location': {'address': 'Toronto',
        'lat': 43.751976046055574,
        'lng': -79.33214044722958,
        'labeledLatLngs': 

In [14]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [15]:
import pandas as pd
venues = results['response']['groups'][0]['items']
    
nearby_venues = pd.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Brookbanks Park,Park,43.751976,-79.332140
1,Variety Store,Food & Drink Shop,43.751974,-79.333114


In [16]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

2 venues were returned by Foursquare.


In [17]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Creating the dataframe toronto_venues

In [18]:
toronto_venues = getNearbyVenues(names=df2['Neighborhood'],
                                   latitudes=df2['Latitude'],
                                   longitudes=df2['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Ontario Provincial Government
Islington Avenue
Malvern, Rouge
Don Mills North
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
The Danforth  East
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmount Park
Bayview Village
Downsview East
The Danforth

In [19]:
print(toronto_venues.shape)
toronto_venues.head()

(4924, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Allwyn's Bakery,43.759840,-79.324719,Caribbean Restaurant
1,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
2,Parkwoods,43.753259,-79.329656,Tim Hortons,43.760668,-79.326368,Café
3,Parkwoods,43.753259,-79.329656,Bruno's valu-mart,43.746143,-79.324630,Grocery Store
4,Parkwoods,43.753259,-79.329656,High Street Fish & Chips,43.745260,-79.324949,Fish & Chips Shop


In [20]:
toronto_venues.groupby("Neighborhood").count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,45,45,45,45,45,45
"Alderwood, Long Branch",27,27,27,27,27,27
"Bathurst Manor, Wilson Heights, Downsview North",32,32,32,32,32,32
Bayview Village,17,17,17,17,17,17
"Bedford Park, Lawrence Manor East",43,43,43,43,43,43
...,...,...,...,...,...,...
"Willowdale, Newtonbrook",30,30,30,30,30,30
Woburn,8,8,8,8,8,8
Woodbine Heights,28,28,28,28,28,28


How many unique categories can be curated from all the returned venues

In [21]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 332 uniques categories.


# Analyze Each Neighborhood

In [22]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Zoo,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,...,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Water Park,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
toronto_onehot.shape

(4924, 332)

 Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [24]:
toronto_grouped=toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Zoo,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,...,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Water Park,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.023256,0.0,...,0.0,0.023256,0.0,0.0,0.0,0.0,0.0,0.023256,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,"Willowdale, Newtonbrook",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
98,Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
99,Woodbine Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
100,York Mills West,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0


In [25]:
toronto_grouped.shape

(102, 332)

Print each neighborhood along with the top 5 most common venues

In [26]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                  venue  freq
0    Chinese Restaurant  0.16
1         Shopping Mall  0.07
2                Bakery  0.04
3  Caribbean Restaurant  0.04
4        Sandwich Place  0.04


----Alderwood, Long Branch----
               venue  freq
0     Discount Store  0.11
1  Convenience Store  0.07
2               Park  0.07
3        Pizza Place  0.07
4           Pharmacy  0.07


----Bathurst Manor, Wilson Heights, Downsview North----
         venue  freq
0  Pizza Place  0.06
1     Pharmacy  0.06
2  Coffee Shop  0.06
3         Park  0.06
4         Bank  0.06


----Bayview Village----
                 venue  freq
0                 Bank  0.12
1  Japanese Restaurant  0.12
2        Grocery Store  0.12
3          Gas Station  0.12
4         Intersection  0.06


----Bedford Park, Lawrence Manor East----
                  venue  freq
0    Italian Restaurant  0.07
1           Coffee Shop  0.07
2            Restaurant  0.05
3  Fast Food Restaurant  0.05
4      Sushi Restaurant  0.05

In [27]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

 Create the new dataframe and display the top 10 venues for each neighborhood

In [60]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Chinese Restaurant,Shopping Mall,Caribbean Restaurant,Bakery,Sandwich Place,Breakfast Spot,Malay Restaurant,Lounge,Seafood Restaurant,Café
1,"Alderwood, Long Branch",Discount Store,Pizza Place,Convenience Store,Pharmacy,Park,Playground,Donut Shop,Shopping Mall,Dance Studio,Intersection
2,"Bathurst Manor, Wilson Heights, Downsview North",Pizza Place,Pharmacy,Bank,Park,Coffee Shop,Mediterranean Restaurant,Mobile Phone Shop,Shopping Mall,Sandwich Place,Supermarket
3,Bayview Village,Grocery Store,Bank,Gas Station,Japanese Restaurant,Restaurant,Park,Trail,Shopping Mall,Skating Rink,Chinese Restaurant
4,"Bedford Park, Lawrence Manor East",Coffee Shop,Italian Restaurant,Bank,Sushi Restaurant,Fast Food Restaurant,Sandwich Place,Restaurant,Pub,Baby Store,Intersection


# Cluster Neighborhoods

In [29]:
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 4, 4, 4, 4, 4, 1])

In [30]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df2

# merge manhattan_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,1.0,Park,Bus Stop,Pharmacy,Shopping Mall,Pizza Place,Fish & Chips Shop,Supermarket,Caribbean Restaurant,Discount Store,Tennis Court
1,M4A,North York,Victoria Village,43.725882,-79.315572,4.0,Coffee Shop,Portuguese Restaurant,Hockey Arena,Sporting Goods Shop,Pizza Place,Men's Store,Gym / Fitness Center,Lounge,Golf Course,Electronics Store
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,4.0,Coffee Shop,Theater,Café,Restaurant,Park,Pub,Diner,Sushi Restaurant,Breakfast Spot,Italian Restaurant
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,4.0,Furniture / Home Store,Fast Food Restaurant,Restaurant,Clothing Store,Coffee Shop,Vietnamese Restaurant,Accessories Store,Sushi Restaurant,Fried Chicken Joint,Dessert Shop
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494,4.0,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Park,Pizza Place,Restaurant,Dance Studio,Breakfast Spot,Thai Restaurant,Middle Eastern Restaurant


In [31]:
toronto_merged['Cluster Labels'].isna().sum()



1

In [33]:
toronto_merged.dropna(inplace=True)

In [34]:
toronto_merged.shape

(102, 16)

In [35]:
map_clusters = folium.Map(location=[tor_lat, tor_lon], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
       color = rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Examine Clusters

Cluster 1

In [55]:
Cluster_1=toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
Cluster_1

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Etobicoke,0.0,Park,Hotel,Pizza Place,Mexican Restaurant,Gym,Theater,Clothing Store,Restaurant,Grocery Store,Bank
12,Scarborough,0.0,Breakfast Spot,Playground,Park,Burger Joint,Italian Restaurant,Yoga Studio,Escape Room,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant
45,North York,0.0,Park,Pool,Yoga Studio,Escape Room,Doner Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
50,North York,0.0,Electronics Store,Bakery,Bank,Park,Italian Restaurant,Arts & Crafts Store,Pharmacy,Shopping Mall,Yoga Studio,Dry Cleaner
57,North York,0.0,Bakery,Discount Store,Convenience Store,Golf Course,Park,Storage Facility,Gas Station,Yoga Studio,Electronics Store,Donut Shop
101,Etobicoke,0.0,Park,Gym / Fitness Center,Shopping Mall,Eastern European Restaurant,Ice Cream Shop,Italian Restaurant,Bus Stop,Yoga Studio,Donut Shop,Dry Cleaner


In [56]:
Cluster_1['1st Most Common Venue'].describe()

count        6
unique       4
top       Park
freq         3
Name: 1st Most Common Venue, dtype: object

Cluster 2

In [48]:
Cluster_2=toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
Cluster_2

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,1.0,Park,Bus Stop,Pharmacy,Shopping Mall,Pizza Place,Fish & Chips Shop,Supermarket,Caribbean Restaurant,Discount Store,Tennis Court
5,Etobicoke,1.0,Pharmacy,Bank,Playground,Convenience Store,Café,Shopping Mall,Golf Course,Bakery,Grocery Store,Skating Rink
6,Scarborough,1.0,Trail,Fast Food Restaurant,Greek Restaurant,Caribbean Restaurant,Bank,Bakery,Supermarket,Chinese Restaurant,Restaurant,Paper / Office Supplies Store
8,East York,1.0,Coffee Shop,Brewery,Pizza Place,Gym / Fitness Center,Construction & Landscaping,Bank,Bakery,Soccer Stadium,Gastropub,Café
10,North York,1.0,Fast Food Restaurant,Grocery Store,Coffee Shop,Pizza Place,Gas Station,Metro Station,Sandwich Place,Sushi Restaurant,Latin American Restaurant,Discount Store
16,York,1.0,Pizza Place,Convenience Store,Coffee Shop,Grocery Store,Bagel Shop,Hockey Arena,Restaurant,Field,Soccer Stadium,Playground
17,Etobicoke,1.0,Coffee Shop,Café,Shopping Plaza,Shopping Mall,Intersection,Gas Station,Farmers Market,Pet Store,Grocery Store,Pharmacy
18,Scarborough,1.0,Restaurant,Pizza Place,Bank,Fast Food Restaurant,Filipino Restaurant,Sports Bar,Beer Store,Sandwich Place,Food & Drink Shop,Convenience Store
21,York,1.0,Pharmacy,Park,Pizza Place,Mexican Restaurant,Discount Store,Japanese Restaurant,Bus Stop,Bank,Coffee Shop,Food Truck
22,Scarborough,1.0,Park,Coffee Shop,Mobile Phone Shop,Fast Food Restaurant,Chinese Restaurant,Indian Restaurant,Yoga Studio,Escape Room,Dry Cleaner,Dumpling Restaurant


In [57]:
Cluster_2['1st Most Common Venue'].describe()

count              39
unique             17
top       Coffee Shop
freq                8
Name: 1st Most Common Venue, dtype: object

Cluster 3

In [49]:
Cluster_3=toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
Cluster_3

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
94,Etobicoke Northwest,2.0,Hotel,Rental Car Location,Coffee Shop,Yoga Studio,Escape Room,Doner Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant


Cluster 4

In [50]:
Cluster_4=toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
Cluster_4

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
53,North York,3.0,Vietnamese Restaurant,Food Truck,Baseball Field,Yoga Studio,Ethiopian Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store


Cluster 5

In [51]:
Cluster_5=toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
Cluster_5

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,4.0,Coffee Shop,Portuguese Restaurant,Hockey Arena,Sporting Goods Shop,Pizza Place,Men's Store,Gym / Fitness Center,Lounge,Golf Course,Electronics Store
2,Downtown Toronto,4.0,Coffee Shop,Theater,Café,Restaurant,Park,Pub,Diner,Sushi Restaurant,Breakfast Spot,Italian Restaurant
3,North York,4.0,Furniture / Home Store,Fast Food Restaurant,Restaurant,Clothing Store,Coffee Shop,Vietnamese Restaurant,Accessories Store,Sushi Restaurant,Fried Chicken Joint,Dessert Shop
4,Queen's Park,4.0,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Park,Pizza Place,Restaurant,Dance Studio,Breakfast Spot,Thai Restaurant,Middle Eastern Restaurant
7,North York,4.0,Japanese Restaurant,Burger Joint,Pizza Place,Coffee Shop,Caribbean Restaurant,Supermarket,Bar,Bank,Liquor Store,Restaurant
9,Downtown Toronto,4.0,Coffee Shop,Café,Japanese Restaurant,Gastropub,Cosmetics Shop,Theater,Italian Restaurant,Hotel,American Restaurant,Creperie
13,North York,4.0,Restaurant,Gym,Coffee Shop,Supermarket,Sushi Restaurant,Discount Store,Shopping Mall,Salon / Barbershop,Intersection,Italian Restaurant
14,East York,4.0,Park,Coffee Shop,Sandwich Place,Café,Pizza Place,Curling Ice,Bus Stop,Intersection,Restaurant,Athletics & Sports
15,Downtown Toronto,4.0,Coffee Shop,Café,Restaurant,Seafood Restaurant,Gastropub,Cosmetics Shop,Theater,Italian Restaurant,American Restaurant,Pizza Place
19,East Toronto,4.0,Coffee Shop,Pub,Beach,Pizza Place,Breakfast Spot,Japanese Restaurant,Burger Joint,Bar,Sandwich Place,Bakery


In [59]:
Cluster_5['1st Most Common Venue'].describe()

count              55
unique             15
top       Coffee Shop
freq               28
Name: 1st Most Common Venue, dtype: object